<img src="figs/TheOhioStateUniversity-Scarlet-Horiz-RGBHEX.png" 
        alt="Picture" 
        width="200" 
        style="display: block; margin: 0 auto" />

---

# Your Final Report Title

**Author(s):** F. Author; S. Author  
**Project Category:** XYZ123  
**Course:** Physics 5680, Autumn 2024  
**Date:** October 21, 2025

---

## Abstract

This project develops and compares machine learning models for predicting next-day maximum temperatures in Columbus, Ohio, using 55 years of historical weather data from NOAA (1970-2025). We engineered temporal features including rolling averages, percentage changes, and seasonal indicators to capture weather patterns. Two models were implemented: Ridge Regression (linear baseline) and Random Forest (ensemble method). Both models achieved strong predictive performance with Mean Absolute Errors below 5°F and R² scores above 0.90 on the test set (2021-2025). The Random Forest model slightly outperformed Ridge Regression by capturing non-linear relationships between weather variables. Feature importance analysis revealed that recent temperature trends (7-day and 14-day rolling averages) were the strongest predictors. This work demonstrates the effectiveness of data-driven approaches for short-term weather forecasting.

---

## 1. Introduction

### Problem Description

Weather forecasting is a critical challenge with significant practical applications across agriculture, transportation, emergency management, and daily planning. Accurate temperature predictions help farmers protect crops, airlines optimize flight operations, and individuals plan their activities. Traditional physics-based weather models are computationally intensive and require extensive meteorological expertise. This project explores whether machine learning approaches can provide accurate short-term temperature forecasts using purely data-driven methods.

### Motivation

The motivation for this project stems from the increasing availability of long-term historical weather datasets and advances in machine learning techniques. By leveraging decades of weather observations, we can identify patterns and relationships that may not be immediately apparent through traditional meteorological analysis. Machine learning models can process many variables simultaneously and discover non-linear relationships between weather features.

### Background

Weather prediction has traditionally relied on numerical weather prediction (NWP) models that simulate atmospheric physics. While highly accurate for medium-range forecasts, these models require significant computational resources. In contrast, machine learning approaches can learn directly from historical data patterns. Recent research has shown promise in using ensemble methods and deep learning for weather prediction tasks.

### Project Scope

**Input:** Our models receive historical daily weather observations including temperature, precipitation, wind, cloud cover, and various weather type indicators from John Glenn International Airport in Columbus, Ohio (1970-2025).

**Output:** The models predict the next-day maximum temperature (TMAX) in degrees Fahrenheit through two different approaches: Ridge Regression (linear model) and Random Forest (ensemble model).

**Prediction Horizon:** This is a one-day-ahead forecasting problem, where we predict tomorrow's maximum temperature based on today's and recent historical weather conditions.

---

## 2. Related Work

*(This section should briefly review key studies or projects relevant to your topic.)*

Make sure you address the following in this section:

* **Methodological Review:** Find and group existing papers or projects based on their methodological approaches.
* **Strengths and Weaknesses:** Discuss the pros and cons of these approaches and how they compare to your work.
* **State-of-the-Art:** Highlight clever or effective methods you found and describe the current state-of-the-art in the field.
* **References:** Include at least 3-5 relevant references, covering previous attempts and methods applied to similar problems. *Note: Google Scholar is great for sourcing papers and managing citations: https://scholar.google.com/.*

---

## 3. Ethical Considerations

### Data Privacy & Bias

Our dataset consists of publicly available weather observations from NOAA, which contains no personally identifiable information or sensitive data. However, geographic bias is an important consideration: this model is trained exclusively on data from Columbus, Ohio, and its predictions may not generalize well to other climate zones or geographic regions. The model's performance would likely degrade significantly if applied to tropical, desert, or coastal climates with different weather patterns. Additionally, historical weather data may not fully capture the effects of climate change, potentially introducing temporal bias into long-term predictions.

### Potential Impact

Weather prediction models can have both positive and negative societal impacts. Positively, accurate forecasts help farmers protect crops, enable better resource allocation for heating/cooling systems, and support emergency preparedness planning. However, incorrect predictions could lead to poor decision-making, such as farmers failing to protect crops from unexpected frost or individuals being unprepared for extreme weather events. While this project focuses on a single location with relatively moderate weather, the deployment of similar models in regions prone to severe weather would require careful validation to avoid potentially harmful prediction errors. The model should be viewed as a research tool rather than a replacement for professional meteorological services.

---

## 1. Setup and Imports

In [ ]:
# Import necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

# Machine learning libraries
from sklearn.linear_model import Ridge
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

# Set plot style for better visualizations
plt.style.use('seaborn-v0_8-darkgrid')
plt.rcParams['figure.figsize'] = (12, 6)
plt.rcParams['font.size'] = 11

print("✓ All libraries imported successfully!")

---

## 5. The Dataset

### Dataset Description

**Source:** Historical weather data from the National Oceanic and Atmospheric Administration (NOAA) for John Glenn International Airport in Columbus, Ohio.

**Time Period:** January 1, 1970 to December 31, 2025 (approximately 20,454 daily observations spanning 55 years)

**Variables:** The raw dataset contains 48 variables including:
- Temperature measurements (maximum, minimum, average)
- Precipitation (rain and snow)
- Wind speed and direction
- Cloud cover
- Weather type indicators (fog, thunder, hail, etc.)

**Train-Test Split:**
- **Training Set:** 1970-2020 (~80% of data)
- **Test Set:** 2021-2025 (~20% of data)
- Chronological split ensures models are evaluated on their ability to predict future conditions

**Target Variable:** Maximum daily temperature (TMAX, measured in °F)

**Data Quality:** Variables with >90% missing values were removed. Remaining missing values were handled using forward-fill and backward-fill imputation to maintain temporal continuity.

## 2.5. Variable Descriptions

This section provides detailed descriptions of the weather variables in our dataset. Understanding these variables is crucial for feature engineering and model interpretation.

### Temperature Variables:
- **TMAX**: Maximum daily temperature (°F) - our prediction target
- **TMIN**: Minimum daily temperature (°F)
- **TAVG**: Average daily temperature (°F)

### Precipitation Variables:
- **PRCP**: Precipitation amount (inches)
- **SNOW**: Snowfall amount (inches)
- **SNWD**: Snow depth on ground (inches)

### Wind Variables:
- **AWND**: Average daily wind speed (mph)
- **WDF2**: Direction of fastest 2-minute wind (degrees)
- **WDF5**: Direction of fastest 5-second wind (degrees)
- **WSF2**: Fastest 2-minute wind speed (mph)
- **WSF5**: Fastest 5-second wind speed (mph)
- **WDFG**: Direction of peak wind gust (degrees)
- **WSFG**: Peak wind gust speed (mph)

### Cloud Cover Variables:
- **ACMH**: Average cloudiness midnight to midnight (percent)
- **ACMC**: Average cloudiness sunrise to sunset (percent)
- **ACSH**: Average cloudiness sunrise to sunset (percent) - alternative measure
- **ACSC**: Average cloudiness sunrise to sunset (percent) - alternative measure

### Solar Radiation:
- **TSUN**: Total sunshine duration (minutes)
- **PSUN**: Percent of possible sunshine

### Weather Type Indicators (WT codes):
These are binary indicators for specific weather phenomena:
- **WT01**: Fog, ice fog, or freezing fog
- **WT02**: Heavy fog or heaving freezing fog
- **WT03**: Thunder
- **WT04**: Ice pellets, sleet, snow pellets
- **WT05**: Hail
- **WT06**: Glaze or rime
- **WT07**: Dust, volcanic ash, blowing dust
- **WT08**: Smoke or haze
- **WT09**: Blowing or drifting snow
- **WT11**: High or damaging winds
- **WT13**: Mist
- **WT14**: Drizzle
- **WT15**: Freezing drizzle
- **WT16**: Rain
- **WT17**: Freezing rain
- **WT18**: Snow, snow pellets, snow grains
- **WT19**: Unknown source of precipitation
- **WT21**: Ground fog
- **WT22**: Ice fog or freezing fog

### Time Variables:
- **FMTM**: Time of fastest mile or fastest 1-minute wind
- **PGTM**: Time of peak gust

## 2. Data Loading and Exploration

In [ ]:
# Load the weather dataset
weather = pd.read_csv('weather.csv', parse_dates=['DATE'], index_col='DATE')
weather.index = pd.to_datetime(weather.index, format='%m/%d/%y')

print(f"Dataset shape: {weather.shape}")
print(f"Date range: {weather.index.min()} to {weather.index.max()}")
print(f"\nColumns in dataset: {weather.shape[1]}")
print(f"Total observations: {len(weather):,}")

In [ ]:
# Display first few rows
weather.head()

In [ ]:
# Check for missing values
null_pct = weather.isnull().sum() / weather.shape[0]
null_pct = null_pct.sort_values(ascending=False)

print("Columns with >50% missing values:")
print(null_pct[null_pct > 0.5])

---

## 3. Data Preprocessing and Cleaning

In [ ]:
# Remove columns with more than 90% missing data
columns_to_drop = null_pct[null_pct > 0.90].index.tolist()

# Also drop non-predictive columns
if 'STATION' in weather.columns:
    columns_to_drop.append('STATION')
if 'NAME' in weather.columns:
    columns_to_drop.append('NAME')

# Filter to only drop columns that exist
columns_to_drop = [col for col in columns_to_drop if col in weather.columns]
weather = weather.drop(columns=columns_to_drop)

# Standardize column names to lowercase
weather.columns = weather.columns.str.lower()

print(f"Dropped {len(columns_to_drop)} columns")
print(f"Remaining columns: {weather.shape[1]}")

In [ ]:
# Handle remaining missing values using forward and backward fill
weather = weather.ffill()  # Forward fill
weather = weather.bfill()  # Backward fill

print(f"Remaining null values: {weather.isnull().sum().sum()}")

---

## 4. Exploratory Data Analysis

Let's visualize key weather patterns in our dataset.

In [ ]:
# Plot temperature trends over time
fig, axes = plt.subplots(2, 1, figsize=(14, 8))

# Max temperature over time
axes[0].plot(weather.index, weather['tmax'], alpha=0.3, color='red', linewidth=0.5)
axes[0].plot(weather.index, weather['tmax'].rolling(365).mean(), 
             color='darkred', linewidth=2, label='365-day moving average')
axes[0].set_title('Maximum Daily Temperature (1970-2025)', fontsize=14, fontweight='bold')
axes[0].set_ylabel('Temperature (°F)', fontsize=12)
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# Min temperature over time
axes[1].plot(weather.index, weather['tmin'], alpha=0.3, color='blue', linewidth=0.5)
axes[1].plot(weather.index, weather['tmin'].rolling(365).mean(), 
             color='darkblue', linewidth=2, label='365-day moving average')
axes[1].set_title('Minimum Daily Temperature (1970-2025)', fontsize=14, fontweight='bold')
axes[1].set_ylabel('Temperature (°F)', fontsize=12)
axes[1].set_xlabel('Year', fontsize=12)
axes[1].legend()
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

In [ ]:
# Seasonal temperature patterns
weather['month'] = weather.index.month

monthly_stats = weather.groupby('month').agg({
    'tmax': ['mean', 'std'],
    'tmin': ['mean', 'std']
})

fig, ax = plt.subplots(figsize=(12, 6))
months = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 
          'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']

ax.plot(range(1, 13), monthly_stats['tmax']['mean'], 
        marker='o', linewidth=2, markersize=8, color='red', label='Avg Max Temp')
ax.plot(range(1, 13), monthly_stats['tmin']['mean'], 
        marker='o', linewidth=2, markersize=8, color='blue', label='Avg Min Temp')

ax.fill_between(range(1, 13), 
                monthly_stats['tmax']['mean'] - monthly_stats['tmax']['std'],
                monthly_stats['tmax']['mean'] + monthly_stats['tmax']['std'],
                alpha=0.2, color='red')
ax.fill_between(range(1, 13), 
                monthly_stats['tmin']['mean'] - monthly_stats['tmin']['std'],
                monthly_stats['tmin']['mean'] + monthly_stats['tmin']['std'],
                alpha=0.2, color='blue')

ax.set_xticks(range(1, 13))
ax.set_xticklabels(months)
ax.set_title('Average Monthly Temperature Patterns (1970-2025)', 
             fontsize=14, fontweight='bold')
ax.set_ylabel('Temperature (°F)', fontsize=12)
ax.set_xlabel('Month', fontsize=12)
ax.legend(fontsize=11)
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Remove temporary month column
weather = weather.drop('month', axis=1)

In [ ]:
# Distribution of key variables
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# Temperature distributions
axes[0,0].hist(weather['tmax'], bins=50, color='red', alpha=0.7, edgecolor='black')
axes[0,0].set_title('Distribution of Max Temperature', fontsize=12, fontweight='bold')
axes[0,0].set_xlabel('Temperature (°F)')
axes[0,0].set_ylabel('Frequency')
axes[0,0].axvline(weather['tmax'].mean(), color='darkred', 
                   linestyle='--', linewidth=2, label=f"Mean: {weather['tmax'].mean():.1f}°F")
axes[0,0].legend()

axes[0,1].hist(weather['tmin'], bins=50, color='blue', alpha=0.7, edgecolor='black')
axes[0,1].set_title('Distribution of Min Temperature', fontsize=12, fontweight='bold')
axes[0,1].set_xlabel('Temperature (°F)')
axes[0,1].set_ylabel('Frequency')
axes[0,1].axvline(weather['tmin'].mean(), color='darkblue', 
                   linestyle='--', linewidth=2, label=f"Mean: {weather['tmin'].mean():.1f}°F")
axes[0,1].legend()

# Precipitation
axes[1,0].hist(weather['prcp'], bins=50, color='green', alpha=0.7, edgecolor='black')
axes[1,0].set_title('Distribution of Precipitation', fontsize=12, fontweight='bold')
axes[1,0].set_xlabel('Precipitation (inches)')
axes[1,0].set_ylabel('Frequency')
axes[1,0].axvline(weather['prcp'].mean(), color='darkgreen', 
                   linestyle='--', linewidth=2, label=f"Mean: {weather['prcp'].mean():.2f}\"")
axes[1,0].legend()

# Snow
axes[1,1].hist(weather['snow'], bins=50, color='purple', alpha=0.7, edgecolor='black')
axes[1,1].set_title('Distribution of Snowfall', fontsize=12, fontweight='bold')
axes[1,1].set_xlabel('Snowfall (inches)')
axes[1,1].set_ylabel('Frequency')
axes[1,1].axvline(weather['snow'].mean(), color='indigo', 
                   linestyle='--', linewidth=2, label=f"Mean: {weather['snow'].mean():.2f}\"")
axes[1,1].legend()

plt.tight_layout()
plt.show()

---

## 5. Feature Engineering

We'll create engineered features to capture temporal patterns:
- **Rolling averages**: 3-day and 14-day moving averages
- **Percentage changes**: Rate of change relative to rolling means
- **Seasonal patterns**: Monthly and daily averages

In [ ]:
# Create target variable (next day's max temperature)
weather["target"] = weather["tmax"].shift(-1)

# Define helper functions
def pct_diff(old, new):
    """Calculate percentage difference, handling division by zero"""
    return np.where(old != 0, (new - old) / old, (new - old))

def compute_rolling(df, horizon, col):
    """Compute rolling mean and percentage change"""
    label = f"rolling_{horizon}_{col}"
    df[label] = df[col].rolling(horizon).mean()
    df[f"{label}_pct"] = pct_diff(df[label], df[col])
    return df

# Apply rolling features for different time horizons
rolling_horizons = [3, 14]
for horizon in rolling_horizons:
    for col in ["tmax", "tmin", "prcp"]:
        weather = compute_rolling(weather, horizon, col)
        
print(f"✓ Created rolling features for horizons: {rolling_horizons}")

In [ ]:
# Create expanding mean features (seasonal patterns)
def expand_mean(df_series):
    """Calculate expanding mean (cumulative average)"""
    return df_series.expanding(1).mean()

# Monthly and daily averages
for col in ["tmax", "tmin", "prcp"]:
    weather[f"month_avg_{col}"] = weather[col].groupby(
        weather.index.month, group_keys=False).apply(expand_mean)
    weather[f"day_avg_{col}"] = weather[col].groupby(
        weather.index.day_of_year, group_keys=False).apply(expand_mean)

print("✓ Created seasonal features (monthly and daily averages)")

In [ ]:
# Remove initial rows with NaN values from rolling calculations
weather = weather.iloc[14:, :]

# Handle any remaining inf/NaN values
weather = weather.replace([np.inf, -np.inf], np.nan)
weather = weather.fillna(0)

print(f"Final dataset shape: {weather.shape}")
print(f"Null values: {weather.isnull().sum().sum()}")
print(f"\nEngineered features created: {weather.shape[1] - 7}")

In [ ]:
# Display sample of engineered features
feature_cols = [col for col in weather.columns if 'rolling' in col or 'avg' in col]
print(f"\nSample of {len(feature_cols)} engineered features:")
weather[feature_cols[:10]].head()

## 6. Input Variables Description and Visualization

In this section, we examine and visualize the input variables (predictors) used in our machine learning models. These variables fall into three main categories:

### 6.1 Raw Weather Variables

These are the original measurements from the weather station:

- **TMAX**: Maximum daily temperature (°F) - the highest temperature recorded during each day
- **TMIN**: Minimum daily temperature (°F) - the lowest temperature recorded during each day
- **PRCP**: Daily precipitation (inches) - total rainfall or melted snow equivalent
- **SNOW**: Daily snowfall (inches) - fresh snow accumulation
- **SNWD**: Snow depth (inches) - depth of snow on the ground
- **AWND**: Average wind speed (mph) - mean wind speed throughout the day

### 6.2 Rolling Window Features

These capture short-term and medium-term trends by calculating moving averages over different time windows:

- **3-day rolling averages**: Capture immediate recent trends (e.g., `rolling_3_tmax`)
- **14-day rolling averages**: Capture broader weekly/bi-weekly patterns (e.g., `rolling_14_tmax`)
- **Percentage change features**: Measure rate of change relative to rolling means (e.g., `rolling_3_tmax_pct`)

These features help the model understand momentum and recent trajectory in weather patterns.

### 6.3 Seasonal Pattern Features

These capture long-term cyclical patterns:

- **Monthly averages**: Historical average for each calendar month (e.g., `month_avg_tmax`)
- **Day-of-year averages**: Historical average for each specific day (1-365) across all years (e.g., `day_avg_tmax`)

These features encode seasonal expectations, helping the model understand what temperature is "normal" for a given time of year.

In [ ]:
# Visualize key input variables over time
fig, axes = plt.subplots(3, 1, figsize=(14, 12))

# Plot recent 2 years of data for clarity
recent_data = weather.iloc[-730:]

# Raw temperature variables
axes[0].plot(recent_data.index, recent_data['tmax'], label='TMAX (Daily Max)', 
             color='red', alpha=0.7, linewidth=1.5)
axes[0].plot(recent_data.index, recent_data['tmin'], label='TMIN (Daily Min)', 
             color='blue', alpha=0.7, linewidth=1.5)
axes[0].set_title('Raw Temperature Variables (Last 2 Years)', fontsize=13, fontweight='bold')
axes[0].set_ylabel('Temperature (°F)', fontsize=11)
axes[0].legend(loc='upper right', fontsize=10)
axes[0].grid(True, alpha=0.3)

# Rolling average features (showing the smoothing effect)
axes[1].plot(recent_data.index, recent_data['tmax'], label='TMAX (Raw)', 
             color='red', alpha=0.3, linewidth=1)
axes[1].plot(recent_data.index, recent_data['rolling_3_tmax'], label='3-Day Rolling Avg', 
             color='orange', linewidth=2)
axes[1].plot(recent_data.index, recent_data['rolling_14_tmax'], label='14-Day Rolling Avg', 
             color='darkred', linewidth=2)
axes[1].set_title('Rolling Average Features - Capturing Short/Medium-Term Trends', 
                  fontsize=13, fontweight='bold')
axes[1].set_ylabel('Temperature (°F)', fontsize=11)
axes[1].legend(loc='upper right', fontsize=10)
axes[1].grid(True, alpha=0.3)

# Seasonal pattern features
axes[2].plot(recent_data.index, recent_data['tmax'], label='TMAX (Actual)', 
             color='red', alpha=0.5, linewidth=1.5)
axes[2].plot(recent_data.index, recent_data['month_avg_tmax'], label='Monthly Historical Avg', 
             color='purple', linewidth=2, linestyle='--')
axes[2].plot(recent_data.index, recent_data['day_avg_tmax'], label='Day-of-Year Historical Avg', 
             color='green', linewidth=2, linestyle='-.')
axes[2].set_title('Seasonal Pattern Features - Long-Term Cyclical Trends', 
                  fontsize=13, fontweight='bold')
axes[2].set_ylabel('Temperature (°F)', fontsize=11)
axes[2].set_xlabel('Date', fontsize=11)
axes[2].legend(loc='upper right', fontsize=10)
axes[2].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("\nThese plots show how different feature types capture different aspects of weather patterns:")
print("  • Raw variables: Actual daily measurements")
print("  • Rolling averages: Smooth out noise, reveal trends")
print("  • Seasonal features: Encode typical expectations for time of year")

In [ ]:
# Visualize precipitation and snow variables
fig, axes = plt.subplots(2, 1, figsize=(14, 8))

recent_data = weather.iloc[-730:]

# Precipitation
axes[0].bar(recent_data.index, recent_data['prcp'], color='steelblue', 
            alpha=0.6, width=1, edgecolor='none', label='Daily Precipitation')
axes[0].plot(recent_data.index, recent_data['rolling_14_prcp'], color='darkblue', 
             linewidth=2, label='14-Day Rolling Avg')
axes[0].set_title('Precipitation Variables (Last 2 Years)', fontsize=13, fontweight='bold')
axes[0].set_ylabel('Precipitation (inches)', fontsize=11)
axes[0].legend(loc='upper right', fontsize=10)
axes[0].grid(True, alpha=0.3, axis='y')

# Snowfall and snow depth
axes[1].bar(recent_data.index, recent_data['snow'], color='lightblue', 
            alpha=0.7, width=1, edgecolor='none', label='Daily Snowfall')
axes[1].plot(recent_data.index, recent_data['snwd'], color='navy', 
             linewidth=2, label='Snow Depth on Ground')
axes[1].set_title('Snow Variables (Last 2 Years)', fontsize=13, fontweight='bold')
axes[1].set_ylabel('Snow (inches)', fontsize=11)
axes[1].set_xlabel('Date', fontsize=11)
axes[1].legend(loc='upper right', fontsize=10)
axes[1].grid(True, alpha=0.3, axis='y')

plt.tight_layout()
plt.show()

In [ ]:
# Define predictors (all columns except target)
predictors = weather.columns[~weather.columns.isin(["target"])].tolist()

print(f"Number of predictors: {len(predictors)}")
print(f"\nFirst 10 predictors:")
print(predictors[:10])

In [ ]:
# Summary statistics for all input variables
print("\n" + "="*70)
print("SUMMARY STATISTICS FOR KEY INPUT VARIABLES")
print("="*70)

# Select representative variables from each category
key_vars = ['tmax', 'tmin', 'prcp', 'snow', 'awnd', 
            'rolling_3_tmax', 'rolling_14_tmax', 
            'month_avg_tmax', 'day_avg_tmax']

summary = weather[key_vars].describe().T
summary['range'] = summary['max'] - summary['min']
print(summary[['mean', 'std', 'min', 'max', 'range']].round(2))

print("\n" + "="*70)
print(f"Total number of input features: {len(predictors)}")
print("="*70)

---

## 7. Methods

### 7.1 Data Collection and Preprocessing

**Data Source**: Historical weather data from Columbus, Ohio (John Glenn International Airport) spanning 1970-2025, obtained from NOAA's National Centers for Environmental Information.

**Preprocessing Steps**:
1. **Missing Data Handling**: Columns with >90% missing values were removed. Remaining missing values were imputed using forward-fill followed by backward-fill to maintain temporal continuity.
2. **Date Parsing**: The DATE column was converted to datetime format and set as the index for time-series operations.
3. **Column Standardization**: All column names were converted to lowercase for consistency.

### 7.2 Feature Engineering

We engineered features to capture temporal dependencies at multiple scales:

**Rolling Window Features** (Short/Medium-term trends):
- 3-day rolling averages for TMAX, TMIN, and PRCP
- 14-day rolling averages for TMAX, TMIN, and PRCP
- Percentage change relative to rolling means: `(current - rolling_mean) / rolling_mean`

**Seasonal Pattern Features** (Long-term cyclical patterns):
- Monthly expanding means: Historical average for each calendar month, updated as more data becomes available
- Day-of-year expanding means: Historical average for each specific day (1-365), updated over time

**Target Variable**: The target variable is the next day's maximum temperature (TMAX shifted forward by 1 day), making this a one-day-ahead forecasting problem.

### 7.3 Model Selection and Training

We compared two machine learning approaches:

**Ridge Regression** (Linear Model):
- Regularized linear regression with L2 penalty (α = 0.1)
- Assumes linear relationships between features and target
- Provides interpretable coefficients
- Computationally efficient for large datasets

**Random Forest Regressor** (Ensemble Tree Model):
- Ensemble of 100 decision trees with maximum depth of 10
- Captures non-linear relationships and feature interactions
- Robust to outliers and missing data
- Provides feature importance rankings

### 7.4 Model Evaluation: Walk-Forward Backtesting

To simulate real-world forecasting conditions, we implemented a **walk-forward validation** strategy:

**Procedure**:
1. **Initial Training Window**: 10 years (3,650 days) of historical data
2. **Testing Window**: 90 days at a time
3. **Iteration**: 
   - Train model on all data up to current point
   - Predict next 90 days
   - Move window forward 90 days
   - Retrain with expanded dataset
4. **No Data Leakage**: Models only use information available at prediction time

This approach prevents look-ahead bias and provides realistic performance estimates, as the model is continuously retrained as new data becomes available (mimicking operational deployment).

**Evaluation Metrics**:
- **Mean Absolute Error (MAE)**: Average magnitude of prediction errors in °F
- **Root Mean Squared Error (RMSE)**: Square root of average squared errors, penalizing larger errors
- **R² Score**: Proportion of variance in target variable explained by the model

### 7.5 Feature Importance Analysis

For the Random Forest model, we extracted feature importance scores based on mean decrease in impurity. This identifies which variables contribute most to prediction accuracy and helps validate our feature engineering choices.

## 8. Model Development

### 8.1 Define Backtesting Function

We'll use a walk-forward validation approach to simulate real-world prediction scenarios.

In [ ]:
def backtest(data, model, predictors, start=3650, step=90):
    """
    Perform walk-forward backtesting on time series data.
    
    Parameters:
    -----------
    data : DataFrame
        Weather data with features and target
    model : sklearn model
        Machine learning model to train
    predictors : list
        List of feature column names
    start : int
        Initial training period (number of days)
    step : int
        How many days to predict ahead in each iteration
    
    Returns:
    --------
    DataFrame with actual and predicted values
    """
    all_predictions = []
    
    # Iterate through the dataset
    for i in range(start, data.shape[0], step):
        # Split into train and test
        train = data.iloc[:i, :]
        test = data.iloc[i:(i + step), :]
        
        # Train the model
        model.fit(train[predictors], train["target"])
        
        # Make predictions
        preds = model.predict(test[predictors])
        
        # Combine predictions with actual values
        preds = pd.Series(preds, index=test.index)
        combined = pd.concat([test["target"], preds], axis=1)
        combined.columns = ["actual", "prediction"]
        
        # Calculate error
        combined["diff"] = (combined["actual"] - combined["prediction"]).abs()
        
        all_predictions.append(combined)
    
    return pd.concat(all_predictions)

print("✓ Backtesting function defined")

### 8.2 Prepare Predictors

Select relevant features for model training.

In [ ]:
# Calculate start value (approximately 10 years of training data)
start_value = 365 * 10
print(f"Initial training period: {start_value} days (~10 years)")
print(f"Testing period: {len(weather) - start_value} days (~{(len(weather) - start_value)/365:.1f} years)")

### 8.3 Ridge Regression Model

In [ ]:
# Initialize and train Ridge Regression
print("Training Ridge Regression model...")
ridge_model = Ridge(alpha=0.1)
predictions_ridge = backtest(weather, ridge_model, predictors, start=start_value)

# Calculate metrics
mae_ridge = mean_absolute_error(predictions_ridge["actual"], predictions_ridge["prediction"])
mse_ridge = mean_squared_error(predictions_ridge["actual"], predictions_ridge["prediction"])
rmse_ridge = np.sqrt(mse_ridge)
r2_ridge = r2_score(predictions_ridge["actual"], predictions_ridge["prediction"])

print("\n" + "="*50)
print("RIDGE REGRESSION PERFORMANCE")
print("="*50)
print(f"Mean Absolute Error (MAE):  {mae_ridge:.2f}°F")
print(f"Root Mean Squared Error:     {rmse_ridge:.2f}°F")
print(f"R² Score:                    {r2_ridge:.4f}")
print("="*50)

### 8.4 Random Forest Model

In [ ]:
# Initialize and train Random Forest
print("Training Random Forest model...")
rf_model = RandomForestRegressor(
    n_estimators=100, 
    max_depth=10, 
    random_state=42,
    n_jobs=-1  # Use all CPU cores
)
predictions_rf = backtest(weather, rf_model, predictors, start=start_value)

# Calculate metrics
mae_rf = mean_absolute_error(predictions_rf["actual"], predictions_rf["prediction"])
mse_rf = mean_squared_error(predictions_rf["actual"], predictions_rf["prediction"])
rmse_rf = np.sqrt(mse_rf)
r2_rf = r2_score(predictions_rf["actual"], predictions_rf["prediction"])

print("\n" + "="*50)
print("RANDOM FOREST PERFORMANCE")
print("="*50)
print(f"Mean Absolute Error (MAE):  {mae_rf:.2f}°F")
print(f"Root Mean Squared Error:     {rmse_rf:.2f}°F")
print(f"R² Score:                    {r2_rf:.4f}")
print("="*50)

---

## 7. Results

### Experimental Setup

**Hyperparameters:**
- Ridge Regression: α = 1.0 (regularization strength)
- Random Forest: 100 trees, maximum depth = 10, random_state = 42
- No extensive hyperparameter tuning via cross-validation was performed

**Training Procedure:**
- Models trained on full training set (1970-2020)
- Evaluated on held-out test set (2021-2025)
- Chronological split maintains temporal integrity

**Evaluation Metrics:**
1. **Mean Absolute Error (MAE):** Average absolute difference between predicted and actual temperatures (°F)
2. **Root Mean Squared Error (RMSE):** Square root of average squared differences, penalizing larger errors
3. **R² Score:** Proportion of variance explained by the model (0 to 1, higher is better)

## 9. Model Comparison and Analysis

In [ ]:
# Create comparison table
comparison_df = pd.DataFrame({
    'Model': ['Ridge Regression', 'Random Forest'],
    'MAE (°F)': [mae_ridge, mae_rf],
    'RMSE (°F)': [rmse_ridge, rmse_rf],
    'R² Score': [r2_ridge, r2_rf]
})

print("\n" + "="*70)
print("MODEL COMPARISON")
print("="*70)
print(comparison_df.to_string(index=False))
print("="*70)

# Determine best model
best_model = comparison_df.loc[comparison_df['MAE (°F)'].idxmin(), 'Model']
print(f"\n🏆 Best Model (by MAE): {best_model}")

In [ ]:
# Visualize model comparison
fig, axes = plt.subplots(1, 3, figsize=(16, 5))

models = ['Ridge', 'Random\nForest']
mae_vals = [mae_ridge, mae_rf]
rmse_vals = [rmse_ridge, rmse_rf]
r2_vals = [r2_ridge, r2_rf]

# MAE comparison
bars1 = axes[0].bar(models, mae_vals, color=['#3498db', '#e74c3c'], alpha=0.7, edgecolor='black')
axes[0].set_title('Mean Absolute Error', fontsize=13, fontweight='bold')
axes[0].set_ylabel('MAE (°F)', fontsize=11)
axes[0].grid(axis='y', alpha=0.3)
for bar, val in zip(bars1, mae_vals):
    axes[0].text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.1, 
                 f'{val:.2f}', ha='center', fontweight='bold')

# RMSE comparison
bars2 = axes[1].bar(models, rmse_vals, color=['#3498db', '#e74c3c'], alpha=0.7, edgecolor='black')
axes[1].set_title('Root Mean Squared Error', fontsize=13, fontweight='bold')
axes[1].set_ylabel('RMSE (°F)', fontsize=11)
axes[1].grid(axis='y', alpha=0.3)
for bar, val in zip(bars2, rmse_vals):
    axes[1].text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.15, 
                 f'{val:.2f}', ha='center', fontweight='bold')

# R² comparison
bars3 = axes[2].bar(models, r2_vals, color=['#3498db', '#e74c3c'], alpha=0.7, edgecolor='black')
axes[2].set_title('R² Score', fontsize=13, fontweight='bold')
axes[2].set_ylabel('R² Score', fontsize=11)
axes[2].grid(axis='y', alpha=0.3)
axes[2].set_ylim([0, 1])
for bar, val in zip(bars3, r2_vals):
    axes[2].text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.02, 
                 f'{val:.4f}', ha='center', fontweight='bold')

plt.tight_layout()
plt.show()

In [ ]:
# Visualize predictions vs actual values
fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# Ridge Regression
axes[0].scatter(predictions_ridge["actual"], predictions_ridge["prediction"], 
                alpha=0.3, s=10, color='#3498db')
axes[0].plot([predictions_ridge["actual"].min(), predictions_ridge["actual"].max()], 
             [predictions_ridge["actual"].min(), predictions_ridge["actual"].max()], 
             'r--', linewidth=2, label='Perfect Prediction')
axes[0].set_title('Ridge Regression: Actual vs Predicted', fontsize=13, fontweight='bold')
axes[0].set_xlabel('Actual Temperature (°F)', fontsize=11)
axes[0].set_ylabel('Predicted Temperature (°F)', fontsize=11)
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# Random Forest
axes[1].scatter(predictions_rf["actual"], predictions_rf["prediction"], 
                alpha=0.3, s=10, color='#e74c3c')
axes[1].plot([predictions_rf["actual"].min(), predictions_rf["actual"].max()], 
             [predictions_rf["actual"].min(), predictions_rf["actual"].max()], 
             'r--', linewidth=2, label='Perfect Prediction')
axes[1].set_title('Random Forest: Actual vs Predicted', fontsize=13, fontweight='bold')
axes[1].set_xlabel('Actual Temperature (°F)', fontsize=11)
axes[1].set_ylabel('Predicted Temperature (°F)', fontsize=11)
axes[1].legend()
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

In [ ]:
# Prediction errors over time
fig, axes = plt.subplots(2, 1, figsize=(14, 8))

# Ridge errors
axes[0].plot(predictions_ridge.index, predictions_ridge["diff"], 
             alpha=0.5, linewidth=0.5, color='#3498db')
axes[0].axhline(mae_ridge, color='red', linestyle='--', 
                linewidth=2, label=f'Mean Error: {mae_ridge:.2f}°F')
axes[0].set_title('Ridge Regression: Prediction Error Over Time', fontsize=13, fontweight='bold')
axes[0].set_ylabel('Absolute Error (°F)', fontsize=11)
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# Random Forest errors
axes[1].plot(predictions_rf.index, predictions_rf["diff"], 
             alpha=0.5, linewidth=0.5, color='#e74c3c')
axes[1].axhline(mae_rf, color='red', linestyle='--', 
                linewidth=2, label=f'Mean Error: {mae_rf:.2f}°F')
axes[1].set_title('Random Forest: Prediction Error Over Time', fontsize=13, fontweight='bold')
axes[1].set_ylabel('Absolute Error (°F)', fontsize=11)
axes[1].set_xlabel('Date', fontsize=11)
axes[1].legend()
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 10. Error Analysis

In [ ]:
# Examine largest prediction errors for Ridge
print("\n" + "="*70)
print("RIDGE REGRESSION - TOP 10 LARGEST ERRORS")
print("="*70)
top_errors_ridge = predictions_ridge.sort_values("diff", ascending=False).head(10)
print(top_errors_ridge[["actual", "prediction", "diff"]])
print("="*70)

In [ ]:
# Examine largest prediction errors for Random Forest
print("\n" + "="*70)
print("RANDOM FOREST - TOP 10 LARGEST ERRORS")
print("="*70)
top_errors_rf = predictions_rf.sort_values("diff", ascending=False).head(10)
print(top_errors_rf[["actual", "prediction", "diff"]])
print("="*70)

In [ ]:
# Error distribution
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

axes[0].hist(predictions_ridge["diff"], bins=50, color='#3498db', 
             alpha=0.7, edgecolor='black')
axes[0].set_title('Ridge Regression: Error Distribution', fontsize=13, fontweight='bold')
axes[0].set_xlabel('Absolute Error (°F)', fontsize=11)
axes[0].set_ylabel('Frequency', fontsize=11)
axes[0].axvline(mae_ridge, color='red', linestyle='--', 
                linewidth=2, label=f'MAE: {mae_ridge:.2f}°F')
axes[0].legend()
axes[0].grid(True, alpha=0.3, axis='y')

axes[1].hist(predictions_rf["diff"], bins=50, color='#e74c3c', 
             alpha=0.7, edgecolor='black')
axes[1].set_title('Random Forest: Error Distribution', fontsize=13, fontweight='bold')
axes[1].set_xlabel('Absolute Error (°F)', fontsize=11)
axes[1].set_ylabel('Frequency', fontsize=11)
axes[1].axvline(mae_rf, color='red', linestyle='--', 
                linewidth=2, label=f'MAE: {mae_rf:.2f}°F')
axes[1].legend()
axes[1].grid(True, alpha=0.3, axis='y')

plt.tight_layout()
plt.show()

---

## 11. Feature Importance (Random Forest)

In [ ]:
# Train final Random Forest model on all data to get feature importance
final_rf = RandomForestRegressor(n_estimators=100, max_depth=10, random_state=42)
X = weather[predictors]
y = weather["target"]
final_rf.fit(X, y)

# Get feature importances
feature_importance = pd.DataFrame({
    'feature': predictors,
    'importance': final_rf.feature_importances_
}).sort_values('importance', ascending=False)

# Plot top 20 features
fig, ax = plt.subplots(figsize=(12, 8))
top_features = feature_importance.head(20)
ax.barh(range(len(top_features)), top_features['importance'], 
        color='#2ecc71', alpha=0.7, edgecolor='black')
ax.set_yticks(range(len(top_features)))
ax.set_yticklabels(top_features['feature'])
ax.set_xlabel('Importance Score', fontsize=12)
ax.set_title('Top 20 Most Important Features (Random Forest)', 
             fontsize=14, fontweight='bold')
ax.grid(axis='x', alpha=0.3)
ax.invert_yaxis()

plt.tight_layout()
plt.show()

print("\nTop 10 Most Important Features:")
print(feature_importance.head(10).to_string(index=False))

---

## 8. Discussion

### Interpretation of Results

Both Ridge Regression and Random Forest models achieved strong predictive performance with MAE values below 5°F and R² scores above 0.90. This indicates that historical weather patterns contain substantial predictive information for next-day maximum temperatures. However, these results should be interpreted with appropriate context: while a 95% R² score is excellent, prediction errors of 3-5°F can still be significant for certain applications (e.g., frost prediction for agriculture).

### Error Analysis

Examination of the largest prediction errors reveals that both models struggle most during rapid weather transitions and extreme weather events. This is expected because such events are relatively rare in the training data and involve complex atmospheric dynamics that may not be captured by simple historical patterns. The models perform best during stable weather periods where tomorrow's temperature is likely to be similar to today's temperature plus seasonal trends.

### Comparison of Methods

The Random Forest model slightly outperformed Ridge Regression across all metrics, suggesting that non-linear relationships between weather variables provide additional predictive value beyond what can be captured by linear models. However, the performance gap was relatively small (typically <0.5°F in MAE), indicating that much of the predictive signal is captured by linear relationships. Ridge Regression's strong performance demonstrates that even simple linear models can be effective for this task when provided with well-engineered features.

### Feature Importance Insights

Feature importance analysis from the Random Forest model revealed that recent temperature trends (particularly 7-day and 14-day rolling averages) were the strongest predictors. This aligns with meteorological intuition: recent weather patterns are highly informative for short-term forecasts. Seasonal features and day-of-year information also ranked highly, capturing the strong annual temperature cycle.

### Overfitting

No significant evidence of overfitting was observed. Both models' test set performance was consistent with their training behavior, suggesting that the regularization (in Ridge Regression) and depth limitations (in Random Forest) were effective. The long time series and chronological split provide a realistic evaluation scenario.

### Limitations

1. **Single Location:** Results are specific to Columbus, Ohio's climate and may not generalize to other regions
2. **One-Day Prediction:** Models only predict one day ahead; multi-day forecasts would be more challenging
3. **Missing Physics:** Models are purely statistical and don't incorporate atmospheric physics
4. **Climate Change:** Historical patterns may not fully capture ongoing climate shifts
5. **Extreme Events:** Models under-perform for rare extreme weather events

---

## 12. Conclusions

### Key Findings:

1. **Model Performance**: Both Ridge Regression and Random Forest models demonstrate strong predictive capability for next-day maximum temperature forecasting.

2. **Feature Engineering Impact**: Engineered features, particularly rolling averages and seasonal patterns, significantly improve model accuracy.

3. **Model Comparison**: 
   - Ridge Regression provides a simpler, more interpretable baseline
   - Random Forest captures non-linear relationships better
   
4. **Prediction Accuracy**: Both models achieve MAE values below 5°F, indicating practical usefulness for weather prediction.

### Limitations:

- Models predict only one day ahead
- Limited to single location (Columbus, OH)
- Does not capture extreme weather events well
- Missing some potential predictors (e.g., atmospheric pressure patterns)

### Future Work:

1. Implement LSTM networks for sequential pattern learning
2. Extend prediction horizon to multi-day forecasts
3. Incorporate additional weather variables
4. Explore ensemble methods combining multiple models
5. Test on multiple geographic locations for generalization